<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Описание-и-подготовка-данных" data-toc-modified-id="Описание-и-подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Описание и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-моделей" data-toc-modified-id="Обучение-и-проверка-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение и проверка моделей</a></span></li><li><span><a href="#Расчет-прибыли" data-toc-modified-id="Расчет-прибыли-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчет прибыли</a></span><ul class="toc-item"><li><span><a href="#Подготовка-к-расчету-прибыли" data-toc-modified-id="Подготовка-к-расчету-прибыли-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Подготовка к расчету прибыли</a></span></li><li><span><a href="#Расчет-прибыли-по-реальным-данным" data-toc-modified-id="Расчет-прибыли-по-реальным-данным-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Расчет прибыли по реальным данным</a></span></li></ul></li><li><span><a href="#Выбор-региона" data-toc-modified-id="Выбор-региона-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выбор региона</a></span></li><li><span><a href="#Итоговые-выводы" data-toc-modified-id="Итоговые-выводы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Итоговые выводы</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Выбор локации для скважины

## Описание проекта

**Заказчик** - добывающая компания «ГлавРосГосНефть». Им необходимо решить, где осуществить бурение новой скважины.

Решение о выборе локации принимается на основании следующих шагов:
* В избранном регионе собираются характеристики для скважин: качество нефти и объём её запасов;
* Строятся модель для предсказания объёма запасов в новых скважинах;
* Выбираются скважины с самыми высокими оценками значений;
* Определяются регион с максимальной суммарной прибылью отобранных скважин.

**Исходные данные** - пробы нефти в трёх регионах: их характеристики и объемы запасов в скважинах. 

Каждый из исходных датафреймов состоит из 5 столбцов, отвечающих за следующие характеристики:
* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

**Ход исследования**

Данное исследование будет состоять из 6 основных этапов, а именно:

1. Описание проекта
2. Описание и подготовка данных
3. Обучение и проверка моделей
4. Расчет прибыли
    * Подготовка к расчету прибыли
    * Расчет прибыли по реальным данным
5. Выбор региона
6. Итоговые выводы

## Описание и подготовка данных

Начнем с общего анализа и описания исходных данных. 

In [2]:
try:
    geo_1 = pd.read_csv('/datasets/geo_data_0.csv')
    geo_2 = pd.read_csv('/datasets/geo_data_1.csv')
    geo_3 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    geo_1 = pd.read_csv('geo_data_0.csv')
    geo_2 = pd.read_csv('geo_data_1.csv')
    geo_3 = pd.read_csv('geo_data_2.csv')

In [3]:
count = 1
for data in [geo_1, geo_2, geo_3]:
    print(f'Инфографика по данным из {count}-го региона')
    print()
    display(data.info())
    display(data.head())
    count += 1

Инфографика по данным из 1-го региона

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Инфографика по данным из 2-го региона

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


Инфографика по данным из 3-го региона

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Что можно сказать по полученным результатам? 

Прежде всего, в данных нет пропусков - количество наблюдений в переменных по всем трем наборам данных совпадает. За исключением одного столбца, остальные имеют тип `float`. Первый столбец, отвечающий за уникальный идентификатор скважины, имеет тип `object` и является слабоинформативным при нашем исследовании. Именно поэтому мы не станем включать данную характеристику в список переменных при моделировании.

Также заранее зададим некоторые параметры, которые пригодятся нам при дальнейшей работе.

In [4]:
RANDOM_STATE = 9

## Обучение и проверка моделей

Перейдем к следующему этапу работы над проектом. Займемся обучением моделей машинного обучения. От заказчика был получен запрос на построение именно линейной регрессии, поскольку остальные модели являются недостаточно предсказуемыми.

Для упрощения работы создадим функцию, которая будет разделять исходную выборку на тренировочную и валидационную, обучать линейную модель, делать предсказания и выводить значимую информацию о результатах.

В качестве итога она будет возвращать датафрейм, состоящий из двух столбцов - фактического объема запасов в скважинах, отобранных для валидационной выборки, а также предсказания от полученных моделей для этих же скважин.

In [5]:
def lin_reg(data):
# Делим выборку на тренировочную и валидационную
    X_train, X_valid, y_train, y_valid = train_test_split(
        data[['f0', 'f1', 'f2']], 
        data['product'],
        test_size=0.25,
        random_state=RANDOM_STATE)
    
# Обучаем линейную регрессию
    reg =  LinearRegression().fit(X_train, y_train)
# Делаем предсказания
    pred = reg.predict(X_valid)
# Объединяем предсказания и фактические величины таргета в датафрейм
    result = pd.DataFrame({'prediction': np.array(pred), 'target': pd.Series(y_valid)})
    print('Средний запас предсказанного сырья:', pred.mean())
    print('Root mean squared error %.2f:' % mean_squared_error(y_valid, pred) ** 0.5)
    return result

Применим полученную функцию к каждому из наборов данных.

In [6]:
print(f'Информация о первом регионе:')
res_geo1 = lin_reg(geo_1)

Информация о первом регионе:
Средний запас предсказанного сырья: 92.45151127970036
Root mean squared error 37.86:


In [7]:
print(f'Информация о втором регионе:')
res_geo2 = lin_reg(geo_2)

Информация о втором регионе:
Средний запас предсказанного сырья: 68.87279592823421
Root mean squared error 0.89:


In [8]:
print(f'Информация о третьем регионе:')
res_geo3 = lin_reg(geo_3)

Информация о третьем регионе:
Средний запас предсказанного сырья: 95.00165908499928
Root mean squared error 40.23:


Что можно сказать о регионах на основании полученной информации?

Первый и третий регионы довольно похожи: они имеют относительно высокое значение средних по региону предсказанных запасов, однако также имеют немаленькую среднеквадратичную ошибку. Это говорит о том, что в среднем запасы в скважинах в каждом из регионов примерно на 40 тыс. баррелей отличаются от полученного нами среднего.

Второй регион, напротив, имеет меньшее значение средних предсказанных запасов сырья, однако значение среднеквадратичной ошибки для данного региона также существенно меньше, что говорит о более точных предсказаниях модели для данного региона.

## Расчет прибыли

### Подготовка к расчету прибыли

Перейдем к расчету потенциальной прибыли от скважин. Однако прежде совершим работу по подготовке к расчетам. Прежде всего, сохраним ключевые для расчетов значения в переменные. Это значительно упростит работу, поскольку при необходимости нам будет необходимо изменить только значения параметров, а не менять дальнейший код.

In [9]:
BUDGET = 10000000
INCOM = 450
WELLS_FINAL = 200
WELLS_FIRST = 500

# Переменные budget и incom указаны в тысячах

Также рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Дополнительно сравним полученный объём сырья со средним запасом в каждом регионе.

In [10]:
breakeven = BUDGET/(INCOM*WELLS_FINAL)
print(f'Для безубыточной разработки новой скважины нужно примерно {int(breakeven)} тыс. баррелей')

Для безубыточной разработки новой скважины нужно примерно 111 тыс. баррелей


Как можно заметить, предсказанные нашими моделями средние объемы запасов сырья в каждом из исследуемых регионов меньше безубыточного объема. Между тем, фактический результат запасов может быть достаточным для безубыточной разработки, а проблема будет заключаться в некачественных предсказаниях наших моделей.

### Расчет прибыли по реальным данным

еперь займемся расчетом прибыли по  выбранным скважинам и предсказаниям модели. Для этого создадим функцию, которая на вход будет принимать датафрейм с предсказаниями и фактическим объемом, а на выход будет возвращать выручку, рассчитанную по 200 потенциально лучшим скважинам - именно столько выбирает компания при разведке региона.

In [11]:
def profit(data, amount):
# Отсортируем входной датафрейм в порядке убывания по значенриям предсказаний
    data = data.sort_values(by='prediction', ascending = False)
# Отберем 200 топовых скважин
    selected = data['target'][data.index][:amount]
# Рассчитаем прибыль
    revenue = selected.sum() * INCOM - BUDGET
    return revenue

Применим полученную функцию к полученным в третьем пункту датафреймам для расчета выручки. 

In [12]:
count = 1
for data in [res_geo1, res_geo2, res_geo3]:
    print(f'Прибыль от полученного сырья в {count}-ом регионе: {profit(data, WELLS_FINAL)}.')
    count += 1

Прибыль от полученного сырья в 1-ом регионе: 3360764.3065996747.
Прибыль от полученного сырья в 2-ом регионе: 2415086.6966815125.
Прибыль от полученного сырья в 3-ом регионе: 2536479.130021617.


Напомним, что полученный нами значения исчисляются в тысячах рублей.

Согласно проведенными нами расчетам, наибольшее значение прибыли достигается в первом регионе - она составляет 3,3 млрд. рублей. Расчетная прибыль во втором и третьем регионах примерно равна и составляется порядка 2,5 млрд.

Однако полученные нами в данном пункте результаты могут быть неточными, поскольку для расчетов прибыли мы используем сильно урезанную выборку - лишь 200 наблюдений из 25 000. Для более объективной оценки проведем дополнительную работу по оценке потенциальной прибыли.

## Выбор региона

Мы уже упоминали ранее, что полученные нами размеры прибыли могут быть несколько не объективными относительно всего набора. Для получения более точного значения прибыли воспользуемся техникой Bootstrap - она создаст заданное количество подвыборок, рассчитает для них прибыль, после чего мы усредним полученные значения и получим более объективные размеры потенциальной прибыли для каждого из регионов. 

Также рассчитаем потенциальные риски и убытки от добычи. По условию заказчика, допустимо оставлять только те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирается итоговый регион на основании наибольшего размера средней прибыли.

In [13]:
state = np.random.RandomState(RANDOM_STATE)

In [14]:
def risk_revenue(data, total, ammount, interval):
    values = []
    for i in range(1000):
        target_subsample = data['target'].sample(n=total, replace=True, random_state=state)
        probs_subsample = data['prediction'][target_subsample.index] 
        new_data = pd.DataFrame({'prediction': pd.Series(probs_subsample), 
                                'target': pd.Series(target_subsample)})
        values.append(profit(new_data, ammount))

    values = pd.Series(values)
    lower = values.quantile(1-interval)
    upper = values.quantile(interval)
    mean = values.mean()
    loss = values.quantile(0.025)
    
    print("Средняя прибыль:", mean)
    print("5%-квантиль:", lower)
    print("95%-квантиль:", upper)
    if loss < 0:
        print(f'С вероятностью 2.5% убыток будет равняться {loss*(-1)} тыс. рублей.')
    else: print(f'С вероятностью 2.5% прибыль будет равняться {loss} тыс. рублей.')
    print(f'Вероятность убытков составляет {(values < 0).mean()*100}%')
    return

In [15]:
count = 1
for data in [res_geo1, res_geo2, res_geo3]:
    print(f'Информация по {count}-ому региону:')
    risk_revenue(data, WELLS_FIRST, WELLS_FINAL, 0.95)
    count += 1
    print('')

Информация по 1-ому региону:
Средняя прибыль: 402307.08634044515
5%-квантиль: -69508.66209429344
95%-квантиль: 858128.7272632724
С вероятностью 2.5% убыток будет равняться 153332.15781436817 тыс. рублей.
Вероятность убытков составляет 7.6%

Информация по 2-ому региону:
Средняя прибыль: 496930.0513132936
5%-квантиль: 138681.653110249
95%-квантиль: 857372.7352301951
С вероятностью 2.5% прибыль будет равняться 57428.456070956076 тыс. рублей.
Вероятность убытков составляет 1.0999999999999999%

Информация по 3-ому региону:
Средняя прибыль: 333483.45528260653
5%-квантиль: -142768.30174572568
95%-квантиль: 806548.5292256572
С вероятностью 2.5% убыток будет равняться 226652.80505422704 тыс. рублей.
Вероятность убытков составляет 13.100000000000001%



Как можно заметить, полученные после применения техники Bootstrap размеры средней прибыли для каждого из регионов существенно изменились. Теперь лидирующую позицию по данному значению занимает второй регион с прибылью порядка 500 млн. В остальных регионах размер средней прибыли несколько меньше - порядка 400 млн. для первого и 340 млн. для третьего.

Обратим также внимание на полученные квантили и вероятности убытков. По итогу две из трех регионов будут иметь убытки с вероятностью не меньше 2.5% - более того, даже 5% квантили у данных регионов имеют отрицательные размеры прибыли. Между тем, второй регион, напротив, имеет прибыль, а не убытки, что делает его единственным регионом, подошедшим под установленные заказчиком критерии. На основании этого (а также того факта, что максимальная средняя прибыль также достигается во втором регионе), мы предлагаем для разработки скважин именно второй регион.  

## Итоговые выводы

Подведем итоги проделанной нами работы. Перед нами стояла задача по выбору наилучшего региона для бурения новой скважины. Изначально нам была предоставлена информация о пробах нефти в трёх регионах. Первичный анализ показал, что исходные данные полные и не нуждаются в существенной доработке.

Нами были построены три модели линейной регрессии - для каждого региона соответственно. На основании предсказаний наших моделей мы рассчитали средний запас в скважинах в каждом регионе и качество полученных моделей. По итогу первый и третий регионы обладали наибольшим средним объемом запасов с 92 и 98 тыс. баррелями, однако также они обладали наибольшим размером среднеквадратичной ошибки - порядка 40 тыс. баррелей. Второй регион, напротив, обладал меньшим размером как среднего объема - 68 тыс. баррелей, так и ошибки - 0.89 тыс. баррелей. Также мы сравнили полученные значения средних объемов каждого из регионов с необходимым объемом для покрытия расходов - средние показатели ни одной из моделей не перешли порог для безубыточности.

Далее мы рассчитали потенциальную фактическую прибыль по 200 лучшим скважинам, отобранным по результатам предсказаний. Лидирующую позицию занял первый регион - 3,3 млрд. рублей, а значение прибыли во втором и третьем регионах было довольно близко - порядка 2,5 млрд. рублей.

Однако после расчета прибыли с применением техники Bootstrap позиции регионов сильно изменились. Первый и третий регионы выбыли из рассмотрения вследствие несоответствия одного из условий заказчика - остаются лишь те регионы, в которых вероятность убытков меньше 2.5%. При этом наибольшей средней прибылью стал обладать именно второй регион - порядка 500 млн. На основании полученных в последнем пункте результатов мы рекомендуем использовать для разработки скважин именно второй регион.